# 算法基础

## 线性回归

### 正规方程法

$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n
$

简化模式  
$
\hat{y} = h_{\mathbf{\theta}}(\mathbf{x}) = \mathbf{\theta}^T \cdot \mathbf{x}
$

MSE损失函数  
$
\text{MSE}(\mathbf{X}, h_{\mathbf{\theta}}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{(\mathbf{\theta}^T \cdot \mathbf{x}^{(i)} - y^{(i)})^2}
$

该损失函数拥有一个闭式解,其正规方程为:  
$
\hat{\mathbf{\theta}} = (\mathbf{X}^T \cdot \mathbf{X})^{-1} \cdot \mathbf{X}^T \cdot \mathbf{y}
$

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]  #在每一个实例的第一列添加 x0 = 1 
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

np.linalg.inv(): 计算一个矩阵的逆矩阵  
A.dot(B) : A矩阵与B矩阵的点乘

当特征数量变得非常庞大时(e.g.,100,000),正规方程的计算时间会变得非常长

### 梯度下降法

线性回归的MSE损失函数是一个凸函数,这表示其中只有全局最小值,而没有局部最小值  
同时,该损失函数还是一个斜率不会突然改变的连续函数  
这两个特征保证了梯度下降法一定能找到全局最小值

### 批量梯度下降(Batch Gradient Descent)

计算损失函数的偏导  
$
\dfrac{\partial}{\partial \theta_j} \text{MSE}(\mathbf{\theta}) = \dfrac{2}{m}\sum\limits_{i=1}^{m}(\mathbf{\theta}^T \cdot \mathbf{x}^{(i)} - y^{(i)})\, x_j^{(i)}
$  
j特征方向上的梯度

当然,在实际使用时,可以一次性计算所有特征的梯度   
$
\nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta}) =
\begin{pmatrix}
 \frac{\partial}{\partial \theta_0} \text{MSE}(\mathbf{\theta}) \\
 \frac{\partial}{\partial \theta_1} \text{MSE}(\mathbf{\theta}) \\
 \vdots \\
 \frac{\partial}{\partial \theta_n} \text{MSE}(\mathbf{\theta})
\end{pmatrix}
 = \dfrac{2}{m} \mathbf{X}^T \cdot (\mathbf{X} \cdot \mathbf{\theta} - \mathbf{y})
$  
其中,梯度向量记为$ \nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta}) $

注:在该公式形式下,在每一步梯度下降时,都计算了一整个训练集$\mathbf{X}$  
因此,当训练集样本数量非常大时,该方法速度非常慢

当得到了梯度向量(指向上升),只要取其相反数便能得到梯度下降值  
$
\mathbf{\theta}^{(\text{next step})} = \mathbf{\theta} - \eta \nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta})
$  
其中,$\eta$表示学习率,学习率太小的话需要迭代太多次达到最优解,学习率太大的话则会跳过最优解并逐渐偏离

In [ ]:
eta = 0.1
n_iterations = 1000
m = 100
theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

另一个可调的参数是迭代次数  
迭代次数太少的话,可能无法达到最优解,迭代次数太多的话,则可能达到最优解后仍然在继续迭代    
一个简单的解决方案是,设置一个较大的迭代次数,中途一旦梯度向量小于tolerance(一个比较小的值)时,则停止迭代

### 随机梯度下降法(Stochastic Gradient Descent)

随机梯度下降法相比普通的梯度下降法更为无序  
在训练的过程中,损失函数时而下降,时而上升,但总体上损失函数仍逐渐下降    
随着不断训练,损失函数将逐渐接近最小值,但无法到达最小值,因此最终得到的参数并不是最优解,而是较为接近最优解的解

正因为随机梯度下降法无序的特性,它能够帮助算法跳出局部最小点,从而拥有更大的机会找到全局最小点

#### 模拟退火法(simulated annealing)

为了能够找到尽可能接近最优解的解,一种解决方案是采用逐渐降低的学习率  
一开始采用较大的值,用以加快训练进程并跳出局部最小点  
之后逐渐降低,最终不断趋近于最优解, 该方法称为模拟退火法

决定每次迭代学习率的函数称为learning schedule

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

In [ ]:
#scikit-learn中的随机梯度下降法
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(n_iter=50, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

n_iter=50 表示运行50次epoch  
penalty=None 表示不适用任何正则化  
eta0=0.1 表示初始学习率为0.1